In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
from scipy.stats import t

In [2]:
df_price = pd.read_csv("preise2023.csv")
df_load = pd.read_csv("load2023.csv")
hours = pd.Series(range(8760))    #csv eingelesen und hours erstellt

df_price.rename(columns={'AT': 'Price'}, inplace=True)
df_load.rename(columns={'load': 'Load'}, inplace=True)  #csv Spaltennamen renamed
df_combined = pd.concat([hours, df_price['Price'], df_load['Load']], axis=1) #daten in ein dataframegepackt 
df_combined.rename(columns={df_combined.columns[0]: 'hours'}, inplace=True) #hours spalte in hours umbenannt
df_combined = df_combined[df_combined['Price'] > 0]     #Preise unter 0 weggecuttet


In [3]:

df_combined['Load_change'] = df_combined['Load'].pct_change()  # Prozentuale Änderung in der Last
df_combined['Price_change'] = df_combined['Price'].pct_change()  # Prozentuale Änderung im Preis
df_combined['Elasticity'] = df_combined['Load_change'] / df_combined['Price_change']  #Elastizität


In [4]:

df_combined['Elasticity'].replace([np.inf, -np.inf], np.nan, inplace=True) #ersetzt unendliche Wert mit nan
df_combined['Elasticity'].fillna(0, inplace=True)  # Ersetzt NaN mit 0
df_combined['Load_change'].fillna(0, inplace=True) 
df_combined['Price_change'].fillna(0, inplace=True)  


C:\Users\peter\AppData\Local\Temp\ipykernel_24572\1401087948.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_combined['Elasticity'].replace([np.inf, -np.inf], np.nan, inplace=True) #ersetzt unendliche Wert mit nan
C:\Users\peter\AppData\Local\Temp\ipykernel_24572\1401087948.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whi

In [5]:
print(df_combined)

      hours  Price    Load  Load_change  Price_change  Elasticity
0         0  1.206  5280.8     0.000000      0.000000    0.000000
9         9  0.029  5884.5     0.114320     -0.975954   -0.117137
10       10  0.073  5891.2     0.001139      1.517241    0.000750
11       11  0.114  5767.5    -0.020997      0.561644   -0.037386
12       12  0.134  5657.8    -0.019020      0.175439   -0.108416
...     ...    ...     ...          ...           ...         ...
8755   8755  1.957  6398.9    -0.054229     -0.253623    0.213816
8756   8756  1.720  6057.6    -0.053337     -0.121104    0.440427
8757   8757  0.820  6091.0     0.005514     -0.523256   -0.010537
8758   8758  1.068  5803.8    -0.047152      0.302439   -0.155904
8759   8759  0.935  5648.8    -0.026707     -0.124532    0.214456

[8623 rows x 6 columns]


#### Nullhypothese: Die Preiselastizität ist nicht statistisch signifikant.

In [6]:
# Preisänderung und Laständerung extrahieren (die entsprechenden Spalten)
x = df_combined['Price_change']  # Unabhängige Variable: Preisänderung
y = df_combined['Load_change']   # Abhängige Variable: Laständerung

# Füge einen konstanten Term (Intercept) hinzu
x = sm.add_constant(x)

# Erstelle das Modell und passe es an die Daten an
model = sm.OLS(y, x).fit()

# Zeige die Zusammenfassung des Modells, um p-Werte zu erhalten
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            Load_change   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     4.473
Date:                Fri, 04 Apr 2025   Prob (F-statistic):             0.0345
Time:                        14:26:03   Log-Likelihood:                 12955.
No. Observations:                8623   AIC:                        -2.591e+04
Df Residuals:                    8621   BIC:                        -2.589e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0014      0.001      2.343   

In [7]:
# Berechnung von t_krit
alpha = 0.05  # Signifikanzniveau
df = 8621  # Freiheitsgrade

t_critical = t.ppf(1 - alpha/2, df)  # Zweiseitiger Test
print(round(t_critical,2))

1.96


#### Preiselastizität ist signifikaten, da der t-wert 2.115 beträgt und somit den Grenzwert von 1.96 überschreitet. Weiters ist der p-wert kleiner 0.5, was bedeutet, dass die Nullhypothese verworfen werden kann. Obwohl der R²-Wert sehr gering ist, ist der Effekt statistisch signifikant - die Tageszeit hat also viel größeren Einfluss auf Last und somit Preis.